In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
import pandas as pd
import numpy as np
from random import randrange
import json

In [3]:
# Connect to the default database
conn = psycopg2.connect("dbname=postgres user=kaiting host=localhost")
conn.autocommit = True
cur = conn.cursor()

# Create database if it doesn't exist
cur.execute("SELECT 1 FROM pg_database WHERE datname = 'yelp'")
if not cur.fetchone():
    cur.execute("CREATE DATABASE yelp")
    print("Database 'yelp' created.")
else:
    print("Database 'yelp' already exists.")

cur.close()
conn.close()

Database 'yelp' already exists.


In [4]:
# Connect to your PostgreSQL database
# Make sure it is already running on your computer.
# (This is done by default on JupyterHub)
# Assuming the database mydb exists...
conn = psycopg2.connect("postgresql://localhost/yelp")

# Create a cursor object
cur = conn.cursor()

# Create a table
cur.execute("""
  CREATE TABLE IF NOT EXISTS users (
    id SERIAL PRIMARY KEY,
    name TEXT,
    age INT
  )
""")

# Insert some data into the table
cur.execute("""
  INSERT INTO users (name, age) VALUES
  ('Alice', 30),
  ('Bob', 25),
  ('Charlie', 35)
""")

# Insert data using safe query interpolation
cur.execute(
  "INSERT INTO users (name, age) VALUES (%s, %s)",
  ('David', 28)
)

# Insert data using a prepared statement
# See also: https://www.postgresql.org/docs/8.1/sql-syntax.html#AEN1368
prepared_statement = "INSERT INTO users (name, age) VALUES (%s, %s)"
cur.execute(prepared_statement, ('Eve', 22))

# Insert data from a list
user_list = [
  ('Frank', 40),
  ('Grace', 29),
  ('Hannah', 33)
]

# Use executemany to insert multiple rows
# This properly escapes quotes, etc.
cur.executemany(
  "INSERT INTO users (name, age) VALUES (%s, %s)",
  user_list
)

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

## Sampling

In [5]:
def reservoir_sample(data, n, k):
  # fill the reservoir array
  r = []
  for i in range(k):
    r.append(data[i])

  # replace elements w/gradually decreasing prob.
  for i in range(k, n):
    # generates a uniform integer between 0 and a-1
    j = randrange(i+1)
    if j < k:
        r[j] = data[i]

  return r

data = list(range(1000))
n, k = len(data), 5
r = reservoir_sample(data, n, k)
r

[721, 92, 262, 794, 769]

In [6]:
df_business = pd.read_json("yelp_academic_dataset_business.json", lines=True)
df_business_dict = df_business.to_dict(orient='records')

In [7]:
print(f"Total rows: {len(df_business)}")
df_business.head()

Total rows: 150346


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [8]:
n = len(df_business)
k = 15000
df_business_sample = reservoir_sample(df_business_dict, n, k)
df_business_sample = pd.DataFrame(df_business_sample)

In [9]:
# Extract business_id from df_business_sampled
business_ids = set(df_business_sample["business_id"])

review_sample = []

with open('yelp_academic_dataset_review.json', 'r') as f:
    for line in f:
        review = json.loads(line)
        if review['business_id'] in business_ids:
            review_sample.append(review)

df_review_sample = pd.DataFrame(review_sample)
df_review_sample.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
1,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06
2,rGQRf8UafX7OTlMNN19I8A,1WHRWwQmZOZDAhp2Qyny4g,uMvVYRgGNXf5boolA9HXTw,5.0,2,0,0,My experience with Shalimar was nothing but wo...,2015-06-21 14:48:06
3,UBp0zWyH60Hmw6Fsasei7w,4Uh27DgGzsp6PqrH913giQ,otQS34_MymijPTdNBoBdCw,4.0,0,2,0,The bun makes the Sonoran Dog. It's like a snu...,2011-10-27 17:12:05
4,-P5E9BYUaK7s3PwBF5oAyg,Jha0USGDMefGFRLik_xFQg,bMratNjTG5ZFEA6hVyr-xQ,5.0,0,0,0,First time there and it was excellent!!! It fe...,2017-02-19 13:32:05


In [10]:
# Extract user_id from df_review_sample
user_ids = set(df_review_sample["user_id"])

user_sample = []

with open('yelp_academic_dataset_user.json', 'r') as f:
    for line in f:
        user = json.loads(line)
        if user['user_id'] in user_ids:
            user_sample.append(review)

df_user_sample = pd.DataFrame(user_sample)
df_user_sample.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,RwcKOdEuLRHNJe4M9-qpqg,6JehEvdoCvZPJ_XIxnzIIw,VAeEXLbEcI9Emt9KGYq9aA,3.0,10,3,7,Located in the 'Walking District' in Nashville...,2018-01-02 22:50:47
1,RwcKOdEuLRHNJe4M9-qpqg,6JehEvdoCvZPJ_XIxnzIIw,VAeEXLbEcI9Emt9KGYq9aA,3.0,10,3,7,Located in the 'Walking District' in Nashville...,2018-01-02 22:50:47
2,RwcKOdEuLRHNJe4M9-qpqg,6JehEvdoCvZPJ_XIxnzIIw,VAeEXLbEcI9Emt9KGYq9aA,3.0,10,3,7,Located in the 'Walking District' in Nashville...,2018-01-02 22:50:47
3,RwcKOdEuLRHNJe4M9-qpqg,6JehEvdoCvZPJ_XIxnzIIw,VAeEXLbEcI9Emt9KGYq9aA,3.0,10,3,7,Located in the 'Walking District' in Nashville...,2018-01-02 22:50:47
4,RwcKOdEuLRHNJe4M9-qpqg,6JehEvdoCvZPJ_XIxnzIIw,VAeEXLbEcI9Emt9KGYq9aA,3.0,10,3,7,Located in the 'Walking District' in Nashville...,2018-01-02 22:50:47


In [11]:
print(f"Total rows: {len(df_user_sample)}")
print(f"Total rows: {len(df_review_sample)}")

Total rows: 406151
Total rows: 670087


In [12]:
df_business_sample.to_json("business_sample.json", orient="records", lines=True)
df_review_sample.to_json("review_sample.json", orient="records", lines=True)
df_user_sample.to_json("user_sample.json", orient="records", lines=True)